In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-04-04 17:52:55,033 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield text
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield text
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield text
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield text
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield text
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield text
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield text
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield text
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [187]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-04-09 13:29:31,848 : INFO : loading Dictionary object from stem100_dictionary
2018-04-09 13:29:31,872 : INFO : loaded stem100_dictionary


In [6]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-04-04 20:59:55,672 : INFO : loaded corpus index from stem100.mm.index
2018-04-04 20:59:55,673 : INFO : initializing corpus reader from stem100.mm
2018-04-04 20:59:55,673 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [53]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


CPU times: user 2h 20min 57s, sys: 22.9 s, total: 2h 21min 20s
Wall time: 2h 21min 56s


In [189]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [190]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [56]:
# %%time
# model = gensim.models.Word2Vec(iter_token_corpus(), size=300, iter = 10, min_count=0, sample=0, workers=8)

2018-04-05 04:54:34,235 : INFO : collecting all words and their counts
2018-04-05 04:54:34,352 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-05 04:54:45,324 : INFO : PROGRESS: at sentence #10000, processed 11872635 words, keeping 28140 word types
2018-04-05 04:54:54,443 : INFO : PROGRESS: at sentence #20000, processed 22389436 words, keeping 31281 word types
2018-04-05 04:55:02,576 : INFO : PROGRESS: at sentence #30000, processed 32260628 words, keeping 32838 word types
2018-04-05 04:55:09,437 : INFO : PROGRESS: at sentence #40000, processed 40518216 words, keeping 34451 word types
2018-04-05 04:55:14,960 : INFO : PROGRESS: at sentence #50000, processed 47137590 words, keeping 35170 word types
2018-04-05 04:55:20,051 : INFO : PROGRESS: at sentence #60000, processed 53235450 words, keeping 36026 word types
2018-04-05 04:55:24,545 : INFO : PROGRESS: at sentence #70000, processed 58570547 words, keeping 36380 word types
2018-04-05 04:55:29,984 : INFO :

2018-04-05 04:58:44,735 : INFO : PROGRESS: at 0.61% examples, 757323 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:45,745 : INFO : PROGRESS: at 0.63% examples, 759048 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:46,747 : INFO : PROGRESS: at 0.65% examples, 760191 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:47,747 : INFO : PROGRESS: at 0.66% examples, 761032 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:48,750 : INFO : PROGRESS: at 0.67% examples, 760539 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:49,764 : INFO : PROGRESS: at 0.70% examples, 761030 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:50,772 : INFO : PROGRESS: at 0.72% examples, 760246 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:51,784 : INFO : PROGRESS: at 0.75% examples, 759865 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:52,790 : INFO : PROGRESS: at 0.77% examples, 760399 words/s, in_qsize 0, out_qsize 0
2018-04-05 04:58:53,792 : INFO : PROGRESS: at 0.79% examples, 759829 words/s, in_q

2018-04-05 05:00:06,293 : INFO : PROGRESS: at 2.95% examples, 754683 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:07,294 : INFO : PROGRESS: at 2.97% examples, 755092 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:08,303 : INFO : PROGRESS: at 2.99% examples, 754996 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:09,310 : INFO : PROGRESS: at 3.01% examples, 755265 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:10,317 : INFO : PROGRESS: at 3.03% examples, 755335 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:11,322 : INFO : PROGRESS: at 3.05% examples, 755537 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:12,336 : INFO : PROGRESS: at 3.08% examples, 755748 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:13,346 : INFO : PROGRESS: at 3.11% examples, 756252 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:14,356 : INFO : PROGRESS: at 3.14% examples, 756742 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:00:15,364 : INFO : PROGRESS: at 3.19% examples, 757382 words/s, in_q

2018-04-05 05:01:27,849 : INFO : PROGRESS: at 5.49% examples, 760282 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:28,879 : INFO : PROGRESS: at 5.51% examples, 760167 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:29,891 : INFO : PROGRESS: at 5.53% examples, 760320 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:30,898 : INFO : PROGRESS: at 5.54% examples, 760497 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:31,899 : INFO : PROGRESS: at 5.56% examples, 760635 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:32,903 : INFO : PROGRESS: at 5.57% examples, 760678 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:33,910 : INFO : PROGRESS: at 5.59% examples, 760658 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:34,921 : INFO : PROGRESS: at 5.61% examples, 760657 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:35,929 : INFO : PROGRESS: at 5.64% examples, 760574 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:01:36,938 : INFO : PROGRESS: at 5.65% examples, 760598 words/s, in_q

2018-04-05 05:02:49,414 : INFO : PROGRESS: at 9.19% examples, 757373 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:50,421 : INFO : PROGRESS: at 9.20% examples, 757377 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:51,436 : INFO : PROGRESS: at 9.21% examples, 757381 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:52,450 : INFO : PROGRESS: at 9.22% examples, 757237 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:53,453 : INFO : PROGRESS: at 9.23% examples, 757129 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:54,459 : INFO : PROGRESS: at 9.24% examples, 757024 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:55,476 : INFO : PROGRESS: at 9.25% examples, 756843 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:56,484 : INFO : PROGRESS: at 9.26% examples, 756773 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:57,489 : INFO : PROGRESS: at 9.27% examples, 756621 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:02:58,503 : INFO : PROGRESS: at 9.28% examples, 756583 words/s, in_q

...

2018-04-05 05:49:49,800 : INFO : PROGRESS: at 90.53% examples, 751074 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:50,814 : INFO : PROGRESS: at 90.55% examples, 751085 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:51,820 : INFO : PROGRESS: at 90.57% examples, 751098 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:52,823 : INFO : PROGRESS: at 90.60% examples, 751100 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:53,829 : INFO : PROGRESS: at 90.62% examples, 751086 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:54,838 : INFO : PROGRESS: at 90.64% examples, 751089 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:55,839 : INFO : PROGRESS: at 90.65% examples, 751096 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:56,854 : INFO : PROGRESS: at 90.66% examples, 751099 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:57,859 : INFO : PROGRESS: at 90.68% examples, 751112 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:49:58,863 : INFO : PROGRESS: at 90.70% examples, 751121 wor

2018-04-05 05:51:10,370 : INFO : PROGRESS: at 92.84% examples, 751240 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:11,373 : INFO : PROGRESS: at 92.87% examples, 751252 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:12,376 : INFO : PROGRESS: at 92.90% examples, 751264 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:13,382 : INFO : PROGRESS: at 92.92% examples, 751270 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:14,385 : INFO : PROGRESS: at 92.95% examples, 751277 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:15,390 : INFO : PROGRESS: at 92.97% examples, 751271 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:16,396 : INFO : PROGRESS: at 92.99% examples, 751265 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:17,399 : INFO : PROGRESS: at 93.01% examples, 751260 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:18,407 : INFO : PROGRESS: at 93.02% examples, 751262 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:51:19,419 : INFO : PROGRESS: at 93.05% examples, 751266 wor

2018-04-05 05:52:30,880 : INFO : PROGRESS: at 95.40% examples, 751581 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:31,882 : INFO : PROGRESS: at 95.41% examples, 751580 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:32,885 : INFO : PROGRESS: at 95.43% examples, 751572 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:33,888 : INFO : PROGRESS: at 95.45% examples, 751564 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:34,889 : INFO : PROGRESS: at 95.47% examples, 751572 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:35,907 : INFO : PROGRESS: at 95.49% examples, 751585 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:36,917 : INFO : PROGRESS: at 95.50% examples, 751590 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:37,922 : INFO : PROGRESS: at 95.51% examples, 751586 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:38,927 : INFO : PROGRESS: at 95.53% examples, 751584 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:52:39,936 : INFO : PROGRESS: at 95.55% examples, 751572 wor

2018-04-05 05:53:51,465 : INFO : PROGRESS: at 99.13% examples, 751515 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:52,478 : INFO : PROGRESS: at 99.14% examples, 751513 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:53,489 : INFO : PROGRESS: at 99.15% examples, 751497 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:54,500 : INFO : PROGRESS: at 99.16% examples, 751496 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:55,501 : INFO : PROGRESS: at 99.17% examples, 751492 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:56,508 : INFO : PROGRESS: at 99.17% examples, 751490 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:57,510 : INFO : PROGRESS: at 99.18% examples, 751486 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:58,522 : INFO : PROGRESS: at 99.19% examples, 751473 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:53:59,539 : INFO : PROGRESS: at 99.20% examples, 751459 words/s, in_qsize 0, out_qsize 0
2018-04-05 05:54:00,540 : INFO : PROGRESS: at 99.21% examples, 751454 wor

CPU times: user 2h 31min 42s, sys: 55.5 s, total: 2h 32min 38s
Wall time: 1h 29s


In [57]:
# model.save('word2vec')

2018-04-05 12:25:11,407 : INFO : saving Word2Vec object under word2vec, separately None
2018-04-05 12:25:11,408 : INFO : not storing attribute syn0norm
2018-04-05 12:25:11,409 : INFO : storing np array 'syn0' to word2vec.wv.syn0.npy
2018-04-05 12:25:11,455 : INFO : storing np array 'syn1neg' to word2vec.syn1neg.npy
2018-04-05 12:25:11,476 : INFO : not storing attribute cum_table
2018-04-05 12:25:11,832 : INFO : saved word2vec


In [59]:
model = gensim.models.Word2Vec.load('word2vec')

2018-04-05 12:25:27,126 : INFO : loading Word2Vec object from word2vec
2018-04-05 12:25:27,272 : INFO : loading wv recursively from word2vec.wv.* with mmap=None
2018-04-05 12:25:27,272 : INFO : loading syn0 from word2vec.wv.syn0.npy with mmap=None
2018-04-05 12:25:27,401 : INFO : setting ignored attribute syn0norm to None
2018-04-05 12:25:27,402 : INFO : loading syn1neg from word2vec.syn1neg.npy with mmap=None
2018-04-05 12:25:27,448 : INFO : setting ignored attribute cum_table to None
2018-04-05 12:25:27,449 : INFO : loaded word2vec


In [105]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

IndentationError: unexpected indent (<ipython-input-105-b0cfacef251c>, line 2)

In [191]:
#preprocess a word
def T(word):
    if word not in en_stop:
        return ps.stem(word)

In [192]:
%%time

import math
from six import iteritems
from six.moves import xrange
idf = {}
for word, freq in dictionary.dfs.iteritems():
    idf[dictionary.id2token[word]] = math.log(dictionary.num_docs) - math.log(freq)


CPU times: user 40 ms, sys: 12 ms, total: 52 ms
Wall time: 35.5 ms


In [193]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [208]:
w2v = model.wv
def vec(document):
    return np.add.reduce([idf[i]*w2v[i] for i in document if i in dictionary.token2id])

In [211]:
%%time
doc_vec = []
for x in iter_token_corpus():
    x = vec(x)
    doc_vec.append(x)

CPU times: user 22min 37s, sys: 7.24 s, total: 22min 44s
Wall time: 22min 44s


In [212]:
from scipy.spatial.distance import cosine
def get_scores(query):
    scores = []
    i = 0
    query = vec(query)
    for x in doc_vec:
        if isinstance(x, np.ndarray):
            score = 1 - cosine(x, query)
        else:
            score = -1000
        scores.append(score)
    return scores

In [213]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        

CPU times: user 2min 20s, sys: 512 ms, total: 2min 21s
Wall time: 2min 20s


In [214]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.4303719699382782, 5230],
  ['ConsumerCourt_DCDRC_106530', '0', 0.21444039046764374, 33285],
  ['ConsumerCourt_DCDRC_107608', '1', 0.45401975512504578, 4072],
  ['ConsumerCourt_DCDRC_114291', '1', 0.5304836630821228, 1467],
  ['ConsumerCourt_DCDRC_114382', '1', 0.52848058938980103, 1520],
  ['ConsumerCourt_DCDRC_118185', '1', 0.66713041067123413, 84],
  ['ConsumerCourt_DCDRC_130318', '0', 0.52215838432312012, 1702],
  ['ConsumerCourt_DCDRC_130570', '1', 0.56025838851928711, 917],
  ['ConsumerCourt_DCDRC_131146', '1', 0.63193374872207642, 214],
  ['ConsumerCourt_DCDRC_131717', '1', 0.61243784427642822, 334],
  ['ConsumerCourt_DCDRC_131741', '1', 0.61177551746368408, 336],
  ['ConsumerCourt_DCDRC_131818', '1', 0.63735765218734741, 189],
  ['ConsumerCourt_DCDRC_131950', '0', 0.55103844404220581, 1053],
  ['ConsumerCourt_DCDRC_131972', '0', 0.64419472217559814, 156],
  ['ConsumerCourt_DCDRC_132932', '0', 0.74282544851303101, 7],
  ['ConsumerCourt

In [229]:
model_name = 'w2v_idf'
# Store results in 1st sheet
wb = Workbook()
dest_filename = model_name + '.xlsx'
ws1 = wb.active
ws1.title = " "
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)


#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [231]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
f2 = pd.ExcelFile('all_results.xlsx')
append_df_to_excel("all_results.xlsx", f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel("all_results.xlsx", f.parse(2), sheet_name= f2.sheet_names[1])

In [226]:
filenames.index('ConsumerCourt_DCDRC_57315' +'.txt')

94617

In [227]:
for x in files_to_tokens(full_filenames[94617]):
    print x

['docdocnoconsumercourtdcdrcdocnotextdistrict', u'consum', u'disput', u'redress', 'forum', 'district', u'consum', u'disput', u'redress', 'forum', 'patiala', 'complaintno', 'cc', 'decidedon', 'kuldeepsingh', u'narind', 'singh', u'resid', 'hno', u'villag', 'rampur', u'sahiw', 'tehsilnabha', 'district', 'patiala', u'complain', u'versu', 'klnissan', 'unit', 'yashodha', u'motor', u'pvtltdvillag', 'dhareri', 'jattan', 'bahadurgarh', 'rajpura', 'road', 'patialathrough', u'manag', 'director', 'nissan', 'motor', 'india', 'pvtltd', 'plot', u'noasipcotindustri', 'park', 'mattur', 'post', 'oragadam', 'sriperumbudur', 'taluk', 'kancheepuram', 'dist', 'tamilnadu', 'india', u'op', u'complaintund', 'section', u'consumerprotect', 'act', 'quorum', u'shdrarorapresid', 'smtneelamgupta', 'member', 'present', u'forth', u'complain', 'smtkusum', u'soodadvoc', u'foropposit', u'parti', u'shgssidhuadvoc', 'order', 'drarora', u'presid', u'thecomplain', u'purchas', 'car', 'make', 'nissan', 'micra', 'xv', 'premium'

In [220]:
for x in files_to_tokens(full_filenames[244922]):
    print x

['docdocnojharkhandhcdocnotextin', 'high', 'court', 'jharkhand', 'ranchi', 'wp', 's', 'nirmal', 'kumar', 'singh', u'petition', u'versusth', 'state', 'jharkhand', u'other', 'respondentscoram', u'honbl', u'mr', u'justic', 'poonam', 'srivastavfor', u'petition', u'mr', 'c', 'prabha', 'advocatefor', u'respond', 'jc', 'sr', 'sc', 'ii', u'learn', 'counsel', u'appear', 'behalf', u'petition', u'state', u'want', 'withdraw', 'writ', u'petitionaccordingli', 'writ', u'petit', u'dismiss', 'withdrawnpoonam', 'srivastav', 'j', 'akscptextdoc']
